# Connecting Tools to our Agent
In this exercise, you'll create the healthcare agent's first tool: **web search**.
This allows it to fetch up-to-date medical information from the internet, not just rely on the model's internal knowledge.

![Agent with web search tool](images/user_tool_agent_answer.png)

### ❗️ Note: Run the **hidden cell** below to initialize the agent, before running the rest of the code. ❗️ 

In [11]:
!pip install -q haystack-ai


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


### 🔐 Configuring API Access

Storing API keys directly in code is unsafe, so we’ll keep our secrets in environment variables managed by DataLab. Here’s the three‑step flow we’ll follow:

1️⃣ Generate a Serper API key

1. Go to https://serper.dev/.
2. Create your account.

3. Click “Create new key.”

<img src="images/serper_api_key.png" width="300">

4. Copy the key.  


2️⃣ Add the key as an environment variable in DataLab

1. Click the “Environment” tab in the notebook toolbar.  
2. Choose “Environment variables”, then click “Add”. 
3. Fill in the fields:  
   * Name: `SERPERDEV_API_KEY`  
   * Value: _paste the key from Step 1_

<img src="images/environment_variable.png" width="500">

4. Click “Save”.


3️⃣ Connect the environment variable to this notebook session

1. In the “Environment variables” list, click the _three dots_ next to `SERPERDEV_API_KEY`.  
2. Choose “Connect”. 
3. Wait a few seconds for the kernel to restart (or restart manually via Run ▸ Restart kernel).  
   When the green dot appears, we’re good to go! ✅


**_Cost Check:_** Serper has a free plan, with enough credits to complete this course, so no payment is required here, just registering and creating the key. 

**_Security Note:_** API keys are passwords—keep them secret. In production, always use secure storage (cloud secret managers, vaults, etc.) rather than environment variables baked into a container image or pushed to Git.

### 🔍 Running a Web Search Using a Haystack Component

Import and initialize the `SerperDevWebSearch` component. This component queries the web and returns a list of results with titles, snippets, links, and metadata as Haystack `Document` objects:

Try it with a sample query:

In [12]:
from haystack.components.websearch import SerperDevWebSearch

search_component = SerperDevWebSearch()
results = search_component.run(query="What are the common side effects of hyaluronic acid?")

Let's look at the output returned by the web search component.
You should see a list of `Document` objects with content and links to real sources.

In [13]:
results["documents"]

[Document(id=0a62787245e0dd1907d4d4bac606ad61e20aea4ae574eb29a473b620846ded91, content: 'pain, bruising, redness, itching, and swelling', meta: {'title': 'Hyaluronic Acid - StatPearls - NCBI Bookshelf', 'link': 'https://www.ncbi.nlm.nih.gov/books/NBK482440/'}),
 Document(id=171415bfc5d3d9144e888d238403c4feb9ef50ed5799df33e93613fd97d8b852, content: 'Research shows that hyaluronic acid is safe to use. Reactions or adverse effects from hyaluronic aci...', meta: {'title': 'Hyaluronic Acid: What It Is, Benefits, How To Use & Side Effects', 'link': 'https://my.clevelandclinic.org/health/articles/22915-hyaluronic-acid', 'date': 'May 4, 2022', 'position': 1}),
 Document(id=5a34b5ffc8189a1c4d9f6eb9ae9c34f3cf00d34568109f4597d821c44f82ce76, content: 'Hardening of skin and connective tissue (scleroderma): Applying hyaluronic acid to the skin might ma...', meta: {'title': 'Hyaluronic Acid - Uses, Side Effects, and More - WebMD', 'link': 'https://www.webmd.com/vitamins/ai/ingredientmono-1062/hyaluro

### 🧰 Converting the Component Into a Tool

Now, wrap the component using `ComponentTool` class.
This converts the component into a tool the agent can reason about and call using function calling.

In [14]:
from haystack.components.websearch import SerperDevWebSearch
from haystack.tools import ComponentTool

search_tool = ComponentTool(component=SerperDevWebSearch())

The `ComponentTool` automatically extracts the tool's **name**, **description**, and **parameters**.
This metadata is what the agent will use when deciding how and when to use this tool.

In [15]:
search_tool.name

'serper_dev_web_search'

In [16]:
print(search_tool.description)


    Uses [Serper](https://serper.dev/) to search the web for relevant documents.

    See the [Serper Dev website](https://serper.dev/) for more details.

    Usage example:
    ```python
    from haystack.components.websearch import SerperDevWebSearch
    from haystack.utils import Secret

    websearch = SerperDevWebSearch(top_k=10, api_key=Secret.from_token("test-api-key"))
    results = websearch.run(query="Who is the boyfriend of Olivia Wilde?")

    assert results["documents"]
    assert results["links"]
    ```
    


In [17]:
search_tool.parameters

{'description': 'Use [Serper](https://serper.dev/) to search the web.',
 'properties': {'query': {'description': 'Search query.', 'type': 'string'}},
 'required': ['query'],
 'type': 'object'}

### 🤖 Creating an Agent with Tool Access
Now, pass the tool to the `Agent` instance.
You'll also give it a new system prompt that reflects its new web-searching capability.

In [18]:
from haystack.components.agents import Agent
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.tools import ComponentTool
from haystack.components.websearch import SerperDevWebSearch

search_tool = ComponentTool(component=SerperDevWebSearch())

agent = Agent(
    chat_generator=OpenAIChatGenerator(model="gpt-4o-mini"),
    system_prompt="""
    You are a helpful AI assistant that has access to internet.
    Keep your answer concise and use the tools that you're provided with to answer the user's questions.
    """,
    tools=[search_tool]
)

### ▶️ Running the Agent with Tool Access
Let's run the agent again with the same query.
This time, the agent should recognize it needs real-world information and trigger a tool call automatically.

In [19]:
from haystack.dataclasses import ChatMessage

result = agent.run(messages=[ChatMessage.from_user("What are the common side effects of hyaluronic acid?")])
print(result['last_message'].text)

Common side effects of hyaluronic acid include:

- Pain at the injection site
- Redness or skin discoloration
- Swelling
- Bruising
- Itching

In some cases, people might experience difficulty moving the treated area or temporary bumps, particularly after injections. While serious reactions are rare, some individuals may also have allergies leading to more significant effects like aching joints or headaches.

For more detailed information, you can refer to [WebMD](https://www.webmd.com/vitamins/ai/ingredientmono-1062/hyaluronic-acid) or [Cleveland Clinic](https://my.clevelandclinic.org/health/articles/22915-hyaluronic-acid).


👏 Great work! You've built your first complete agent!
This agent can reason about its task, choose the right tool, fetch real-world data, and respond accordingly.